【Task 6】 模型融合（2天）
模型融合方式任意，并结合Task5给出你的最优结果。  
时间：2天  
例如Stacking融合，用你目前评分最高的模型作为基准模型，和其他模型进行stacking融合，得到最终模型及评分结果。

### 导入数据

In [1]:
import pickle
with open('feature.pkl','rb') as f:
    X = pickle.load(f, encoding = 'gbk')

In [2]:
with open('label.pkl','rb') as f:
    y = pickle.load(f)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)

#### 特征归一化TODO


In [4]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train = std.fit_transform(X_train.values)
X_test = std.transform(X_test.values)

### 模型评估

In [5]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

### 模型融合-stacking
选择简单的lr为次级学习器， 其他的svm, dt, rf,xgb 作为初级学习器

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
lr = LogisticRegression(C = 0.1, penalty = 'l1')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8020 测试集： 0.7983
[auc值] 训练集： 0.8179 测试集： 0.7797


In [17]:
lr: 0.7983/0.7797

In [9]:
svm_sigmoid=svm.SVC(kernel = 'sigmoid',probability = True, C = 0.01)
svm_sigmoid.fit(X_train,y_train)
model_metrics(svm_sigmoid, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7378 测试集： 0.7699
[auc值] 训练集： 0.8064 测试集： 0.7654


svm: 0.7699/0.7654

In [10]:
dt = DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19, random_state =2018)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7794 测试集： 0.7665
[auc值] 训练集： 0.7777 测试集： 0.7262


dt: 0.7665/0.7262

In [11]:
rf = RandomForestClassifier(n_estimators=160, max_depth=9, min_samples_split=50,
                            min_samples_leaf=20, max_features = 9,oob_score=True, random_state=2018)
rf.fit(X_train, y_train)
print('袋外分数：', rf.oob_score_)
model_metrics(rf, X_train, X_test, y_train, y_test)

袋外分数： 0.793400286944046
[准确率] 训练集： 0.8311 测试集： 0.7874
[auc值] 训练集： 0.9052 测试集： 0.7649


RF: 0.7874/0.7649

In [12]:
xgb = XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=5, 
                                                  min_child_weight=4, gamma=0.0, subsample=0.65, reg_alpha=1e-05,
                                                  colsample_bytree=0.5, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27)
xgb.fit(X_train, y_train)

model_metrics(xgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8526 测试集： 0.7958
[auc值] 训练集： 0.9094 测试集： 0.7656


xgb: 0.7958/0.7656

### 开始融合

In [20]:
LR = LogisticRegression()

基学习器输出的类别标签作为新特征

In [18]:
from mlxtend.classifier import StackingCVClassifier
sclf = StackingCVClassifier(classifiers=[ svm_sigmoid, dt, rf,lr, xgb], meta_classifier=lr)
sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8425 测试集： 0.7992
[auc值] 训练集： 0.7704 测试集： 0.6777


predit_proba作为新特征

In [21]:
sclf = StackingCVClassifier(classifiers=[svm_sigmoid,xgb, dt, lr, rf], 
                            meta_classifier=LR, use_probas=True)
sclf.fit(X_train, y_train)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8361 测试集： 0.7933
[auc值] 训练集： 0.8868 测试集： 0.7753


### 结果汇总TODO

In [16]:
model = {}